In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:16pt;}
div.text_cell_render.rendered_html{font-size:16pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:16pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
table.dataframe{font-size:16px;}
</style>
"""))

In [4]:
import os # 모델 저장할 폴더가 있는지 여부와 폴더 생성
import numpy as np
import pandas as pd # get_dummies(원핫인코딩), crosstab
import seaborn as sns # iris 데이터(데이터프레임) 가져오기
from sklearn import datasets # iris 데이터(X, y) 가져오기
from sklearn.preprocessing import LabelEncoder # 라벨인코더
from tensorflow.keras.utils import to_categorical # 원핫인코딩 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, LeakyReLU
from tensorflow.keras import metrics # compile
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

# 1. 기본적인 DNN

In [5]:
# 1. 데이터 생성 및 전처리
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y,
                                                   test_size=0.2,
                                                   stratify=iris_y)
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [6]:
# 2. 모델구성
model = Sequential([
    Input(4),
    Dense(units=64, activation='relu'),
    Dense(units=128, activation='relu'),
    Dense(units=50, activation='relu'),
    Dense(units=3, activation='softmax')    
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 50)                6450      
                                                                 
 dense_3 (Dense)             (None, 3)                 153       
                                                                 
Total params: 15,243
Trainable params: 15,243
Non-trainable params: 0
_________________________________________________________________


In [7]:
# 3. 학습과정 설정
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
# 4. 모델 학습
earlystopping = EarlyStopping(patience=40)# monitor='val_loss' 기본값
hist=model.fit(train_X,train_y,
               epochs=200,
               validation_split=0.1,
               callbacks=[earlystopping])

Epoch 1/200
4/4 [==============================] - 1s 61ms/step - loss: 1.1541 - accuracy: 0.3704 - val_loss: 1.0710 - val_accuracy: 0.4167
Epoch 2/200
4/4 [==============================] - 0s 9ms/step - loss: 0.9829 - accuracy: 0.5833 - val_loss: 0.8813 - val_accuracy: 0.8333
Epoch 3/200
4/4 [==============================] - 0s 10ms/step - loss: 0.9131 - accuracy: 0.6296 - val_loss: 0.8016 - val_accuracy: 0.8333
Epoch 4/200
4/4 [==============================] - 0s 9ms/step - loss: 0.8628 - accuracy: 0.6481 - val_loss: 0.7580 - val_accuracy: 0.8333
Epoch 5/200
4/4 [==============================] - 0s 9ms/step - loss: 0.7998 - accuracy: 0.6574 - val_loss: 0.7522 - val_accuracy: 0.8333
Epoch 6/200
4/4 [==============================] - 0s 10ms/step - loss: 0.7433 - accuracy: 0.7870 - val_loss: 0.7189 - val_accuracy: 0.5000
Epoch 7/200
4/4 [==============================] - 0s 10ms/step - loss: 0.6891 - accuracy: 0.6852 - val_loss: 0.6655 - val_accuracy: 0.5833
Epoch 8/200
4/4 [======

4/4 [==============================] - 0s 10ms/step - loss: 0.1029 - accuracy: 0.9722 - val_loss: 0.0235 - val_accuracy: 1.0000
Epoch 60/200
4/4 [==============================] - 0s 10ms/step - loss: 0.1327 - accuracy: 0.9537 - val_loss: 0.0490 - val_accuracy: 1.0000
Epoch 61/200
4/4 [==============================] - 0s 9ms/step - loss: 0.1308 - accuracy: 0.9352 - val_loss: 0.1693 - val_accuracy: 0.9167
Epoch 62/200
4/4 [==============================] - 0s 10ms/step - loss: 0.1165 - accuracy: 0.9352 - val_loss: 0.0321 - val_accuracy: 1.0000
Epoch 63/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0958 - accuracy: 0.9630 - val_loss: 0.0254 - val_accuracy: 1.0000
Epoch 64/200
4/4 [==============================] - 0s 10ms/step - loss: 0.1004 - accuracy: 0.9722 - val_loss: 0.0725 - val_accuracy: 1.0000
Epoch 65/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0881 - accuracy: 0.9537 - val_loss: 0.0521 - val_accuracy: 1.0000
Epoch 66/200
4/4 [=========

4/4 [==============================] - 0s 8ms/step - loss: 0.0732 - accuracy: 0.9537 - val_loss: 0.0585 - val_accuracy: 1.0000
Epoch 118/200
4/4 [==============================] - 0s 8ms/step - loss: 0.0673 - accuracy: 0.9722 - val_loss: 0.0101 - val_accuracy: 1.0000
Epoch 119/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0753 - accuracy: 0.9722 - val_loss: 0.0168 - val_accuracy: 1.0000
Epoch 120/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0678 - accuracy: 0.9722 - val_loss: 0.0381 - val_accuracy: 1.0000
Epoch 121/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0666 - accuracy: 0.9815 - val_loss: 0.0140 - val_accuracy: 1.0000
Epoch 122/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0677 - accuracy: 0.9815 - val_loss: 0.0242 - val_accuracy: 1.0000
Epoch 123/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0640 - accuracy: 0.9815 - val_loss: 0.0334 - val_accuracy: 1.0000
Epoch 124/200
4/4 [=====

4/4 [==============================] - 0s 9ms/step - loss: 0.0579 - accuracy: 0.9815 - val_loss: 0.0276 - val_accuracy: 1.0000
Epoch 176/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0616 - accuracy: 0.9815 - val_loss: 0.0269 - val_accuracy: 1.0000
Epoch 177/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0590 - accuracy: 0.9907 - val_loss: 0.0168 - val_accuracy: 1.0000
Epoch 178/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0646 - accuracy: 0.9815 - val_loss: 0.0251 - val_accuracy: 1.0000
Epoch 179/200
4/4 [==============================] - 0s 8ms/step - loss: 0.0574 - accuracy: 0.9815 - val_loss: 0.0167 - val_accuracy: 1.0000
Epoch 180/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0614 - accuracy: 0.9815 - val_loss: 0.0169 - val_accuracy: 1.0000
Epoch 181/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0587 - accuracy: 0.9907 - val_loss: 0.0147 - val_accuracy: 1.0000
Epoch 182/200
4/4 [=====

# 2. sklearn 이용
- 원핫인코딩을 하지 않고 라벨인코딩까지만 해야 작동

In [8]:
from sklearn.neural_network import MLPClassifier

In [9]:
# 1. 데이터
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [10]:
model = MLPClassifier(
            hidden_layer_sizes=(64, 128, 50), # hidden layer의 units 수
            activation='relu',
            solver='adam', # sgd등의 optimizer
            batch_size=40,
            max_iter=1000, # 학습 최대 횟수
            early_stopping=True, # 조기 종료 활성화
            n_iter_no_change=20, # earlyStopping의 patience와 유사
            validation_fraction=0.1, # 검증셋 비율
            warm_start=False # True일 경우 이전학습에 이어서 학습
)

In [11]:
model.fit(train_X, train_y)

MLPClassifier(batch_size=40, early_stopping=True,
              hidden_layer_sizes=(64, 128, 50), max_iter=1000,
              n_iter_no_change=20)

In [12]:
model.score(test_X, test_y)

0.9

In [13]:
iris_X[0]

array([5.1, 3.5, 1.4, 0.2])

In [14]:
# 모델 사용
input_data = [[5.1, 3.5, 1.4, 0.2]]
model.predict(input_data)

array([0])

In [15]:
# 교차표
# test_y : 실제값
hat_y = model.predict(test_X)
pd.crosstab

<function pandas.core.reshape.pivot.crosstab(index, columns, values=None, rownames=None, colnames=None, aggfunc=None, margins: 'bool' = False, margins_name: 'str' = 'All', dropna: 'bool' = True, normalize=False) -> 'DataFrame'>

# 3. 클래스를 생성하여 모델 생성함수 사용

In [16]:
class DNNClassifier:
    @staticmethod
    def build(input_dim=4, activation='relu', output_dim=3):
#         model = Sequential()
#         model.add(Input(input_dim))
#         model.add(Dense(units=50, activation=activation))
#         model.add(Dense(units=30, activation=activation))
#         model.add(Dense(units=3, activation='softmax'))
        # 모델구성
        model = Sequential([
            Input(input_dim),
            Dense(units=50, activation=activation),
            Dense(units=30, activation=activation),
            Dense(units=output_dim, activation='softmax')
        ])
        # 학습 설정
        model.compile(loss='sparse_categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])
        return model

In [17]:
# 1. 데이터
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
# 2. 모델
model = DNNClassifier.build(input_dim=4, activation='elu')
# 3. 모델 학습
hist = model.fit(train_X, train_y, epochs=50, validation_split=0.1, verbose=0)

(120, 4) (120,) (30, 4) (30,)


In [18]:
# 모델 평가
loss, accuracy = model.evaluate(test_X, test_y)
print(f'정확도 : {accuracy*100} %')

1/1 [==============================] - 0s 20ms/step - loss: 0.1769 - accuracy: 1.0000
정확도 : 100.0 %


# 4. 함수형 API
- 병렬처리, Residual block

In [19]:
# 기존 model 스타일1
model = Sequential()
model.add(Input(shape=(4,)))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=30, activation='relu'))
model.add(Dense(units=3, activation='softmax'))
# 기존 model 스타일2
model = Sequential([
    Input(shape=(4,)),
    Dense(units=50, activation='relu'),
    Dense(units=30, activation='relu'),
    Dense(units=3, activation='softmax')
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 50)                250       
                                                                 
 dense_11 (Dense)            (None, 30)                1530      
                                                                 
 dense_12 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [20]:
# 기존 model 스타일3 (함수형 API)
from tensorflow.keras import Model
input_ = Input(shape=(4,))
layer1 = Dense(units=50, activation='relu')(input_)
layer2 = Dense(units=30, activation='relu')(layer1)
output = Dense(units=3, activation='softmax')(layer2)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense_13 (Dense)            (None, 50)                250       
                                                                 
 dense_14 (Dense)            (None, 30)                1530      
                                                                 
 dense_15 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [22]:
# 병렬처리 방식
from tensorflow.keras.layers import concatenate
input_ = Input(shape=(4,))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=80, activation='relu')(input_)
dense3 = Dense(units=30, activation='relu')(input_)
x = concatenate([dense1, dense2, dense3])
output = Dense(units=3, activation='softmax')(x)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 dense_20 (Dense)               (None, 50)           250         ['input_7[0][0]']                
                                                                                                  
 dense_21 (Dense)               (None, 80)           400         ['input_7[0][0]']                
                                                                                                  
 dense_22 (Dense)               (None, 30)           150         ['input_7[0][0]']                
                                                                                            

In [23]:
# 리지듀얼 블얼(Residual Block)
    #딥러닝에서 기울기 소실 문제로 학습이 잘 되지 않는 부분을 해결하기 위한 제안
from tensorflow.keras.layers import add
input_ = Input(shape=(4, ))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=50, activation='relu')(dense1)
dense3 = add([dense1, dense2])
output = Dense(units=3, activation='softmax')(dense3)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 dense_24 (Dense)               (None, 50)           250         ['input_8[0][0]']                
                                                                                                  
 dense_25 (Dense)               (None, 50)           2550        ['dense_24[0][0]']               
                                                                                                  
 add (Add)                      (None, 50)           0           ['dense_24[0][0]',               
                                                                  'dense_25[0][0]']         

In [24]:
# 학습과정 설정 & 학습
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=100, verbose=0)

In [25]:
# 학습 평가
loss, accuracy = model.evaluate(test_X, test_y, verbose=0)
print(f'accuracy : {accuracy}')

accuracy : 0.9666666388511658
